# Day 7: No Space Left On Device

In [87]:
open System.Text.RegularExpressions

let extract pattern content =
    Regex.Matches(content, pattern)
    |> Seq.map (fun m -> m.Value)

let (|Match|_|) (pattern: string) (value: string) = 
    if (Regex.IsMatch(value, pattern)) then
        Some (extract pattern value)
    else 
        None

let (|Prefix|_|) (pattern: string) (value: string) = 
    if (value.StartsWith(pattern)) then 
        Some (value.Remove(0, pattern.Length)) 
    else 
        None
        
let applyRight f (first, second) =
    (first, f second)

In [88]:
let getCurrentDirectory (current:string) command =
    match command with
    | "$ cd /" -> "/"
    | "$ cd .." -> current.Substring(0, current.LastIndexOf("/") + 1)
    | Prefix "$ cd " directory -> current + directory + "/"
    | _ -> current
    
let addFiles directory fileSystem file =
    match file with
    | Match "^\d+" _ -> (directory, file) :: fileSystem
    | Prefix "dir " folder -> (directory, directory + folder) :: fileSystem
    | _ -> fileSystem

In [89]:
open System.Collections.Generic

let folderSize (fileSystem: IDictionary<string, seq<int>>) folder =
    fileSystem.Item folder
    |> Seq.sum

let fileSize fileSystem file =
    match file with
    | Match "^\d+" matches -> Seq.head matches |> int
    | Prefix "^dir " folder -> (folderSize fileSystem folder)
    | _ -> 0

In [91]:
let rec createFileSystem currentDirectory fileSystem commands =
    match commands with
    | [] -> fileSystem
    | command::rest -> 
        let directory = getCurrentDirectory currentDirectory command 
        createFileSystem directory (addFiles directory fileSystem command) rest

let fs =
    File.ReadAllLines("Day7 - Test.txt")
    |> Seq.toList
    |> createFileSystem "" []
    |> Seq.ofList

fs 
    |> Seq.groupBy fst
    |> Seq.map (applyRight (Seq.map snd))

index,Item1,Item2
0,/a/e/d/,"[ 7214296 k, 5626152 d.ext, 8033020 d.log, 4060174 j ]"
1,/a/e/,[ 584 i ]
2,/a/,"[ 62596 h.lst, 2557 g, 29116 f, /a/e ]"
3,/,"[ /d, 8504156 c.dat, 14848514 b.txt, /a ]"
